In [1]:
from __future__ import print_function

from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam

from sklearn.cluster import KMeans

import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
# build model
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

# build encoder
encoder = Model(input=input_img, output=encoded)

# build autoencoder
adam = Adam(lr=5e-4)
autoencoder = Model(input=input_img, output=decoded)
autoencoder.compile(optimizer=adam, loss='mse')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_6 (Dense)              (None, 784)               101136    
Total para

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


In [3]:
# load images
train_num = 130000
X = np.load('image.npy')
X = X.astype('float32') / 255.
X = np.reshape(X, (len(X), -1))
x_train = X[:train_num]
x_val = X[train_num:]
x_train.shape, x_val.shape

((130000, 784), (10000, 784))

In [4]:
# train autoencoder
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_val, x_val))

Train on 130000 samples, validate on 10000 samples
Epoch 1/50
130000/130000 [==============================] - 2s - loss: 0.0579 - val_loss: 0.0355
Epoch 2/50
130000/130000 [==============================] - 1s - loss: 0.0323 - val_loss: 0.0297
Epoch 3/50
130000/130000 [==============================] - 1s - loss: 0.0279 - val_loss: 0.0260
Epoch 4/50
130000/130000 [==============================] - 1s - loss: 0.0246 - val_loss: 0.0231
Epoch 5/50
130000/130000 [==============================] - 1s - loss: 0.0222 - val_loss: 0.0213
Epoch 6/50
130000/130000 [==============================] - 1s - loss: 0.0206 - val_loss: 0.0200
Epoch 7/50
130000/130000 [==============================] - 1s - loss: 0.0194 - val_loss: 0.0189
Epoch 8/50
130000/130000 [==============================] - 1s - loss: 0.0185 - val_loss: 0.0183
Epoch 9/50
130000/130000 [==============================] - 1s - loss: 0.0177 - val_loss: 0.0175
Epoch 10/50
130000/130000 [==============================] - 1s - loss: 0.01

In [5]:
# after training, use encoder to encode image, and feed it into Kmeans
encoded_imgs = encoder.predict(X)
encoded_imgs = encoded_imgs.reshape(encoded_imgs.shape[0], -1)
kmeans = KMeans(n_clusters=2, random_state=0).fit(encoded_imgs)

In [6]:
# get test cases
f = pd.read_csv('test_case.csv')
IDs, idx1, idx2 = np.array(f['ID']), np.array(f['image1_index']), np.array(f['image2_index'])

In [7]:
# predict
o = open('prediction.csv', 'w')
o.write("ID,Ans\n")
for idx, i1, i2 in zip(IDs, idx1, idx2):
    p1 = kmeans.labels_[i1]
    p2 = kmeans.labels_[i2]
    if p1 == p2:
        pred = 1  # two images in same cluster
    else: 
        pred = 0  # two images not in same cluster
    o.write("{},{}\n".format(idx, pred))
o.close()